# Initialization Library

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Get Base Data

In [ ]:
# Load the recipes dataset from the provided URL
base_data_url = 'https://raw.githubusercontent.com/pijarpahlawan/WasteToTaste/master/recipes.csv'
recipes = pd.read_csv(base_data_url)
recipes.head()

,Title,Ingredients,Steps,Loves,URL
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,/id/resep/4473027-ayam-woku-manado
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,/id/resep/4471956-ayam-goreng-tulang-lunak
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,/id/resep/4473057-ayam-cabai-kawin
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,/id/resep/4473023-ayam-geprek
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,/id/resep/4427438-minyak-ayam


In [ ]:
# Display basic information about the dataset
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        159 non-null    object
 1   Ingredients  159 non-null    object
 2   Steps        159 non-null    object
 3   Loves        159 non-null    int64 
 4   URL          159 non-null    object
dtypes: int64(1), object(4)
memory usage: 6.3+ KB


# Scrape Image

In [ ]:
# Set the base URL for the web scraping
base_url = 'https://cookpad.com'
recipes['URL'] = recipes['URL'].apply(lambda url: base_url + url)
recipes.head()

,Title,Ingredients,Steps,Loves,URL
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,https://cookpad.com/id/resep/4473027-ayam-woku...
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,https://cookpad.com/id/resep/4471956-ayam-gore...
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,https://cookpad.com/id/resep/4473057-ayam-caba...
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,https://cookpad.com/id/resep/4473023-ayam-geprek
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,https://cookpad.com/id/resep/4427438-minyak-ayam


In [ ]:
# Function to scrape images from the recipe URLs
def scrap_images(df):
    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        print(f"Progress: {index/df.shape[0]*100}%")
        headers = {
            "accept": "*/*",
            "accept-language": "en-US,en;q=0.7",
            "content-type": "application/x-www-form-urlencoded",
            "cookie": "your_cookie_data_here",
            "origin": "https://cookpad.com",
            "priority": "u=1, i",
            "referer": "https://cookpad.com/id/homepage",
            "sec-ch-ua": "\"Chromium\";v=\"124\", \"Brave\";v=\"124\", \"Not-A.Brand\";v=\"99\"",
            "sec-ch-ua-mobile": "?1",
            "sec-ch-ua-platform": "\"Android\"",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "sec-gpc": "1",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
        }

        # Make an HTTP request to the webpage
        try:
            response = requests.get(row['URL'], headers=headers)
            response.raise_for_status()  # Ensure no HTTP errors
        except requests.exceptions.RequestException as e:
            print(f"Error fetching the URL: {e}")
            continue

        # Wait for a second before making the next request
        time.sleep(1)

        # Parse the webpage with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the image tag with the specified alt text
        img_tag = soup.find('img', alt=f"{row['Title']} foto resep utama")

        # If the image tag is found, update the DataFrame
        if img_tag:
            img_url = img_tag['src']
            df.at[index, 'IMG URL'] = img_url
        else:
            print("No image with the specified alt text found.")

    return df

In [ ]:
# Scrape images for the recipes
new_recipes = scrap_images(recipes)

Progres: 0.0%
Progres: 0.628930817610063%
Progres: 1.257861635220126%
Progres: 1.8867924528301887%
Progres: 2.515723270440252%
Progres: 3.1446540880503147%
Progres: 3.7735849056603774%
Error fetching the URL: 403 Client Error: Forbidden for url: https://cookpad.com/id/resep/4437475-ayam-saus-telur-asin
No image with the specified alt text found.
Progres: 4.40251572327044%
Progres: 5.031446540880504%
Progres: 5.660377358490567%
Progres: 6.289308176100629%
Error fetching the URL: 403 Client Error: Forbidden for url: https://cookpad.com/id/resep/4472541-mie-ayam-homemade-by-suami
No image with the specified alt text found.
Progres: 6.918238993710692%
Progres: 7.547169811320755%
Progres: 8.176100628930817%
Progres: 8.80503144654088%
Progres: 9.433962264150944%
Progres: 10.062893081761008%
Progres: 10.69182389937107%
Progres: 11.320754716981133%
Progres: 11.949685534591195%
Progres: 12.578616352201259%
Progres: 13.20754716981132%
Progres: 13.836477987421384%
Progres: 14.465408805031446%
Pro

In [ ]:
# Display the head of the updated DataFrame
new_recipes.head()

,Title,Ingredients,Steps,Loves,URL,IMG URL
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,https://cookpad.com/id/resep/4473027-ayam-woku...,https://img-global.cpcdn.com/recipes/4a0bb9e71...
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,https://cookpad.com/id/resep/4471956-ayam-gore...,https://img-global.cpcdn.com/recipes/d820435f5...
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,https://cookpad.com/id/resep/4473057-ayam-caba...,https://img-global.cpcdn.com/recipes/94c448501...
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,https://cookpad.com/id/resep/4473023-ayam-geprek,https://img-global.cpcdn.com/recipes/be1dca0c3...
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,https://cookpad.com/id/resep/4427438-minyak-ayam,https://img-global.cpcdn.com/recipes/c32bfa0b4...


In [ ]:
# Separate the recipes with and without image URLs
success_new_recipes1 = new_recipes.dropna(subset=['IMG URL'])
success_new_recipes1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 0 to 27
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        26 non-null     object
 1   Ingredients  26 non-null     object
 2   Steps        26 non-null     object
 3   Loves        26 non-null     int64 
 4   URL          26 non-null     object
 5   IMG URL      26 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.4+ KB


In [ ]:
unsuccess_new_recipes1 = new_recipes[new_recipes['IMG URL'].isna()]
unsuccess_new_recipes1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, 6 to 158
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        133 non-null    object
 1   Ingredients  133 non-null    object
 2   Steps        133 non-null    object
 3   Loves        133 non-null    int64 
 4   URL          133 non-null    object
 5   IMG URL      0 non-null      object
dtypes: int64(1), object(5)
memory usage: 7.3+ KB


In [ ]:
# Attempt to scrape images again for the unsuccessful recipes
new_recipes2 = scrap_images(unsuccess_new_recipes1)

Progres: 4.511278195488721%
Error fetching the URL: 403 Client Error: Forbidden for url: https://cookpad.com/id/resep/4437475-ayam-saus-telur-asin
No image with the specified alt text found.
Progres: 7.518796992481203%
Error fetching the URL: 403 Client Error: Forbidden for url: https://cookpad.com/id/resep/4472541-mie-ayam-homemade-by-suami
No image with the specified alt text found.
Progres: 21.052631578947366%
Progres: 21.804511278195488%
Progres: 22.55639097744361%
Progres: 23.308270676691727%
Progres: 24.06015037593985%
Progres: 24.81203007518797%
Progres: 25.563909774436087%
Progres: 26.31578947368421%
Error fetching the URL: 404 Client Error: Not Found for url: https://cookpad.com/id/resep/4472671-balado-ikan-sarden
No image with the specified alt text found.
Progres: 27.06766917293233%
Progres: 27.819548872180448%
Error fetching the URL: 403 Client Error: Forbidden for url: https://cookpad.com/id/resep/4472300-sarden-kangkung
No image with the specified alt text found.
Progres:

In [ ]:
# Display information about the newly scraped recipes
new_recipes2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, 6 to 158
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        133 non-null    object
 1   Ingredients  133 non-null    object
 2   Steps        133 non-null    object
 3   Loves        133 non-null    int64 
 4   URL          133 non-null    object
 5   IMG URL      119 non-null    object
dtypes: int64(1), object(5)
memory usage: 11.3+ KB


In [ ]:
# Separate the successful and unsuccessful recipes after the second scrape
success_new_recipes2 = new_recipes2.dropna(subset=['IMG URL'])
success_new_recipes2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119 entries, 28 to 158
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        119 non-null    object
 1   Ingredients  119 non-null    object
 2   Steps        119 non-null    object
 3   Loves        119 non-null    int64 
 4   URL          119 non-null    object
 5   IMG URL      119 non-null    object
dtypes: int64(1), object(5)
memory usage: 6.5+ KB


In [ ]:
# Concatenate the successful recipes from both scrapes
result1 = pd.concat([success_new_recipes1, success_new_recipes2], ignore_index=True)
result1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        145 non-null    object
 1   Ingredients  145 non-null    object
 2   Steps        145 non-null    object
 3   Loves        145 non-null    int64 
 4   URL          145 non-null    object
 5   IMG URL      145 non-null    object
dtypes: int64(1), object(5)
memory usage: 6.9+ KB


In [ ]:
# Display the final result
result1

,Title,Ingredients,Steps,Loves,URL,IMG URL
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,https://cookpad.com/id/resep/4473027-ayam-woku...,https://img-global.cpcdn.com/recipes/4a0bb9e71...
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,https://cookpad.com/id/resep/4471956-ayam-gore...,https://img-global.cpcdn.com/recipes/d820435f5...
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,https://cookpad.com/id/resep/4473057-ayam-caba...,https://img-global.cpcdn.com/recipes/94c448501...
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,https://cookpad.com/id/resep/4473023-ayam-geprek,https://img-global.cpcdn.com/recipes/be1dca0c3...
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,https://cookpad.com/id/resep/4427438-minyak-ayam,https://img-global.cpcdn.com/recipes/c32bfa0b4...


In [ ]:
# Print a specific value from the DataFrame
specific_value = result1.loc[0, 'IMG URL']
print(specific_value)

https://img-global.cpcdn.com/recipes/4a0bb9e71ee86944/680x482cq70/ayam-woku-manado-foto-resep-utama.jpg


In [ ]:
# Save the final result to a CSV file
result1.to_csv('complete_recipe_data.csv', sep=',', index=False, header=True, encoding='utf-8')

In [ ]:
# Load the saved CSV file to verify
data = pd.read_csv('/content/complete_recipe_data.csv')
data.head()

,Title,Ingredients,Steps,Loves,URL,IMG URL
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,https://cookpad.com/id/resep/4473027-ayam-woku...,https://img-global.cpcdn.com/recipes/4a0bb9e71...
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,https://cookpad.com/id/resep/4471956-ayam-gore...,https://img-global.cpcdn.com/recipes/d820435f5...
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,https://cookpad.com/id/resep/4473057-ayam-caba...,https://img-global.cpcdn.com/recipes/94c448501...
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,https://cookpad.com/id/resep/4473023-ayam-geprek,https://img-global.cpcdn.com/recipes/be1dca0c3...
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,https://cookpad.com/id/resep/4427438-minyak-ayam,https://img-global.cpcdn.com/recipes/c32bfa0b4...
